In [31]:
edges = read.csv('../data/edgelist_2011.csv')
nodes = read.csv('../data/nodelist_2011.csv')

In [44]:
head(nodes, 3)

,country_iso3,colonizer,inflation_rate,area,gdp_us_dollar,langoff_1,continent,gdp_growth,landlocked
,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>,<chr>,<dbl>,<int>
1,AFG,USA,11.804186,652225,17804292964,Persian,Asia,0.4263548,1
2,AGO,PRT,13.482468,1246700,111789686464,Portuguese,Africa,3.4719763,0
3,ALB,NONE,3.429123,28748,12890765324,Albanian,Europe,2.5453218,0


In [47]:
n_countries = dim(nodes)[1]
dyads = matrix(0, nrow = n_countries, ncol = n_countries)
nodecovs = matrix(0, nrow = n_countries, ncol = n_countries)
numerical_columns = c('inflation_rate', 'gdp_us_dollar', 'gdp_growth')
categorical_columns = c('colonizer', 'langoff_1', 'landlocked', 'continent')

In [33]:
for (i in 1:n_countries) {
    for (j in 1:n_countries) {
        if (sum(edges$source == nodes$country_iso3[i] & edges$target == nodes$country_iso3[j])) {
            dyads[i, j] = 1
        }
        for (k in 1:length(numerical_columns)) {
            dyad
        }
    }
}

In [42]:
dyads.flatten = as.vector(t(dyads))